# Question 1 – NN Basics

## I. Fitting Data


In [ ]:
using Random
using LinearAlgebra
using Plots


In [ ]:
Random.seed!(123)

n = 500
x = range(0, 2π, length = n)
x_vec = collect(x)

ϵ = 0.1 .* randn(n)
y = sin.(x_vec) .+ ϵ

X = reshape(x_vec, 1, :)
Y = reshape(y, 1, :)


In [ ]:
# Funciones de activación
function activate(z, act)
    if act == :logistic
        return 1 ./(1 .+ exp.(-z))
    elseif act == :tanh
        return tanh.(z)
    elseif act == :relu
        return max.(0, z)
    else
        error("Activación desconocida")
    end
end

# Derivadas de las activaciones
function activate_prime(z, act)
    if act == :logistic
        s = 1 ./(1 .+ exp.(-z))
        return s .* (1 .- s)
    elseif act == :tanh
        t = tanh.(z)
        return 1 .- t.^2
    elseif act == :relu
        return (z .> 0) .* 1.0
    else
        error("Activación desconocida")
    end
end


In [ ]:
# Estructura de la red con 3 capas ocultas (50 neuronas cada una)
mutable struct SimpleNN
    W1::Array{Float64,2}
    b1::Array{Float64,2}
    W2::Array{Float64,2}
    b2::Array{Float64,2}
    W3::Array{Float64,2}
    b3::Array{Float64,2}
    W4::Array{Float64,2}
    b4::Array{Float64,2}
end

# Constructor
function SimpleNN(input_dim::Int, hidden_dim::Int, output_dim::Int)
    W1 = 0.1 .* randn(hidden_dim, input_dim)
    b1 = zeros(hidden_dim, 1)

    W2 = 0.1 .* randn(hidden_dim, hidden_dim)
    b2 = zeros(hidden_dim, 1)

    W3 = 0.1 .* randn(hidden_dim, hidden_dim)
    b3 = zeros(hidden_dim, 1)

    W4 = 0.1 .* randn(output_dim, hidden_dim)
    b4 = zeros(output_dim, 1)

    return SimpleNN(W1, b1, W2, b2, W3, b3, W4, b4)
end


In [ ]:
# Propagación hacia adelante
function forward(nn::SimpleNN, X, acts::NTuple{3,Symbol})
    # Capa 1
    z1 = nn.W1 * X .+ nn.b1
    a1 = activate(z1, acts[1])

    # Capa 2
    z2 = nn.W2 * a1 .+ nn.b2
    a2 = activate(z2, acts[2])

    # Capa 3
    z3 = nn.W3 * a2 .+ nn.b3
    a3 = activate(z3, acts[3])

    # Capa de salida (lineal)
    z4 = nn.W4 * a3 .+ nn.b4
    ŷ = z4

    cache = (z1, a1, z2, a2, z3, a3)
    return ŷ, cache
end


In [ ]:
# Entrenamiento por descenso de gradiente
using Statistics

function train!(nn::SimpleNN, X, Y, acts::NTuple{3,Symbol}; epochs::Int = 500, lr::Float64 = 0.01)
    n = size(X, 2)

    for epoch in 1:epochs
        ŷ, (z1, a1, z2, a2, z3, a3) = forward(nn, X, acts)

        # Gradiente capa de salida
        δ4 = (2 / n) .* (ŷ .- Y)   # 1 × n
        dW4 = δ4 * a3'
        db4 = sum(δ4, dims = 2)

        # Backprop capas ocultas
        δ3 = (nn.W4' * δ4) .* activate_prime(z3, acts[3])
        dW3 = δ3 * a2'
        db3 = sum(δ3, dims = 2)

        δ2 = (nn.W3' * δ3) .* activate_prime(z2, acts[2])
        dW2 = δ2 * a1'
        db2 = sum(δ2, dims = 2)

        δ1 = (nn.W2' * δ2) .* activate_prime(z1, acts[1])
        dW1 = δ1 * X'
        db1 = sum(δ1, dims = 2)

        # Actualización de pesos
        nn.W4 .-= lr .* dW4
        nn.b4 .-= lr .* db4
        nn.W3 .-= lr .* dW3
        nn.b3 .-= lr .* db3
        nn.W2 .-= lr .* dW2
        nn.b2 .-= lr .* db2
        nn.W1 .-= lr .* dW1
        nn.b1 .-= lr .* db1

        if epoch % 100 == 0
            loss = mean((ŷ .- Y).^2)
            @show epoch loss
        end
    end

    return nn
end


In [ ]:
# Dimensiones de la red
input_dim = 1
hidden_dim = 50
output_dim = 1


In [ ]:
# Activación logística
acts_logistic = (:logistic, :logistic, :logistic)
nn_logistic = SimpleNN(input_dim, hidden_dim, output_dim)
train!(nn_logistic, X, Y, acts_logistic; epochs = 600, lr = 0.01)
ŷ_logistic, _ = forward(nn_logistic, X, acts_logistic)
pred_logistic = vec(ŷ_logistic)


In [ ]:
# Activación tanh
acts_tanh = (:tanh, :tanh, :tanh)
nn_tanh = SimpleNN(input_dim, hidden_dim, output_dim)
train!(nn_tanh, X, Y, acts_tanh; epochs = 600, lr = 0.01)
ŷ_tanh, _ = forward(nn_tanh, X, acts_tanh)
pred_tanh = vec(ŷ_tanh)


In [ ]:
# Activación ReLU
acts_relu = (:relu, :relu, :relu)
nn_relu = SimpleNN(input_dim, hidden_dim, output_dim)
train!(nn_relu, X, Y, acts_relu; epochs = 600, lr = 0.01)
ŷ_relu, _ = forward(nn_relu, X, acts_relu)
pred_relu = vec(ŷ_relu)


In [ ]:
# Activación mixta
acts_mixed = (:relu, :tanh, :logistic)
nn_mixed = SimpleNN(input_dim, hidden_dim, output_dim)
train!(nn_mixed, X, Y, acts_mixed; epochs = 600, lr = 0.01)
ŷ_mixed, _ = forward(nn_mixed, X, acts_mixed)
pred_mixed = vec(ŷ_mixed)


In [ ]:
# Gráfico NN con activación logística
p1 = scatter(x_vec, y, label = "Data",
             title = "NN con activación logística",
             xlabel = "x", ylabel = "y")
plot!(p1, x_vec, pred_logistic, label = "Fitted (logistic)", linewidth = 3)
savefig(p1, "plot_logistic_Julia.png")
display(p1)


In [ ]:
# Gráfico NN con activación tanh
p2 = scatter(x_vec, y, label = "Data",
             title = "NN con activación tanh",
             xlabel = "x", ylabel = "y")
plot!(p2, x_vec, pred_tanh, label = "Fitted (tanh)", linewidth = 3)
savefig(p2, "plot_tanh_Julia.png")
display(p2)


In [ ]:
# Gráfico NN con activación ReLU
p3 = scatter(x_vec, y, label = "Data",
             title = "NN con activación ReLU",
             xlabel = "x", ylabel = "y")
plot!(p3, x_vec, pred_relu, label = "Fitted (ReLU)", linewidth = 3)
savefig(p3, "plot_relu_Julia.png")
display(p3)


In [ ]:
# Gráfico NN con activaciones mixtas
p4 = scatter(x_vec, y, label = "Data",
             title = "NN con activaciones mixtas (ReLU, tanh, logistic)",
             xlabel = "x", ylabel = "y")
plot!(p4, x_vec, pred_mixed, label = "Fitted (mixed)", linewidth = 3)
savefig(p4, "plot_mixta_Julia.png")
display(p4)


Visualmente, la activación tanh es la que produce el ajuste más cercano a la forma real de los datos. Esto se debe a que tanh es suave, no saturante cerca del cero, y permite capturar parte de la curvatura del patrón observado. La activación logística genera un ajuste casi plano, lo cual es esperable porque la sigmoide satura rápidamente y pierde sensibilidad ante variaciones de entrada. La activación ReLU muestra una estructura por tramos lineales, lo que limita su capacidad para representar patrones curvos y continuos. La red con activaciones mixtas mejora ligeramente el desempeño, pero no alcanza la suavidad ni la estabilidad del modelo basado únicamente en tanh.


## II. Learning rate

Es un hiperparámetro que controla qué tan grandes son los pasos que da la red neuronal cuando ajusta sus pesos durante el entrenamiento. Un learning rate muy pequeño hace que el aprendizaje avance lentamente y puede que la red no llegue a capturar patrones relevantes. Por su lado, un learning rate muy grande puede provocar inestabilidad, oscilaciones o incluso que la red no aprenda nada. Por eso es importante probar distintos valores y observar cómo cambia el ajuste.


In [ ]:
# Activaciones (versión compacta)
function activate(z, act)
    act == :tanh     && return tanh.(z)
    act == :logistic && return 1 ./(1 .+ exp.(-z))
    act == :relu     && return max.(0, z)
    error("Activación desconocida")
end

function activate_prime(z, act)
    act == :tanh && return 1 .- tanh.(z).^2
    if act == :logistic
        s = 1 ./(1 .+ exp.(-z))
        return s .* (1 .- s)
    end
    act == :relu && return (z .> 0) .* 1.0
    error("Activación desconocida")
end


In [ ]:
# Estructura general para k capas ocultas
mutable struct NN2
    W::Vector{Array{Float64,2}}
    b::Vector{Array{Float64,2}}
end

function build_nn2(input_dim, hidden_dim, output_dim, num_hidden)
    W = Vector{Array{Float64,2}}()
    b = Vector{Array{Float64,2}}()

    # Primera capa
    push!(W, 0.1 .* randn(hidden_dim, input_dim))
    push!(b, zeros(hidden_dim, 1))

    # Capas ocultas adicionales
    for _ in 2:num_hidden
        push!(W, 0.1 .* randn(hidden_dim, hidden_dim))
        push!(b, zeros(hidden_dim, 1))
    end

    # Capa de salida
    push!(W, 0.1 .* randn(output_dim, hidden_dim))
    push!(b, zeros(output_dim, 1))

    return NN2(W, b)
end


In [ ]:
# Forward para NN2
function forward2(nn::NN2, X, act)
    Z = []
    A = [X]

    # Capas ocultas
    for ℓ in 1:length(nn.W)-1
        z = nn.W[ℓ] * A[end] .+ nn.b[ℓ]
        push!(Z, z)
        push!(A, activate(z, act))
    end

    # Capa de salida (lineal)
    zL = nn.W[end] * A[end] .+ nn.b[end]
    push!(Z, zL)
    push!(A, zL)

    return A, Z
end


In [ ]:
# Entrenamiento para NN2
function train2!(nn::NN2, X, Y, act; epochs=300, lr=0.01)
    n = size(X, 2)

    for epoch in 1:epochs
        A, Z = forward2(nn, X, act)
        ŷ = A[end]

        δ = (2/n) .* (ŷ .- Y)

        dW = Vector{Array{Float64,2}}(undef, length(nn.W))
        db = Vector{Array{Float64,2}}(undef, length(nn.W))

        for ℓ in reverse(1:length(nn.W))
            dW[ℓ] = δ * A[ℓ]'
            db[ℓ] = sum(δ, dims = 2)

            if ℓ > 1
                δ = (nn.W[ℓ]' * δ) .* activate_prime(Z[ℓ-1], act)
            end
        end

        for ℓ in 1:length(nn.W)
            nn.W[ℓ] .-= lr .* dW[ℓ]
            nn.b[ℓ] .-= lr .* db[ℓ]
        end
    end

    return nn
end


In [ ]:
# Re-simulación de los datos (mismo DGP)
Random.seed!(123)
n = 500
x = range(0, 2π, length=n)
x_vec = collect(x)
ϵ = 0.1 .* randn(n)
y = sin.(x_vec) .+ ϵ
X = reshape(x_vec, 1, :)
Y = reshape(y, 1, :)


In [ ]:
# 1 hidden layer, distintos learning rates
learning_rates = [0.0001, 0.001, 0.01, 0.1]
preds_1 = Dict{Float64,Vector{Float64}}()

for lr in learning_rates
    nn = build_nn2(1, 50, 1, 1)  # 1 hidden layer
    train2!(nn, X, Y, :tanh; epochs=500, lr=lr)
    A, _ = forward2(nn, X, :tanh)
    preds_1[lr] = vec(A[end])
end


In [ ]:
p1 = scatter(x_vec, y, label="Data",
             title="1 Hidden Layer - Diferentes Learning Rates",
             xlabel="x", ylabel="y")

for lr in learning_rates
    plot!(p1, x_vec, preds_1[lr], label="lr=$(lr)", linewidth=3)
end

savefig(p1, "learningrate_1HL_Julia.png")
display(p1)


In [ ]:
# 2 hidden layers
preds_2 = Dict{Float64,Vector{Float64}}()

for lr in learning_rates
    nn = build_nn2(1, 50, 1, 2)
    train2!(nn, X, Y, :tanh; epochs=500, lr=lr)
    A, _ = forward2(nn, X, :tanh)
    preds_2[lr] = vec(A[end])
end

p2 = scatter(x_vec, y, label="Data",
             title="2 Hidden Layers - Diferentes Learning Rates",
             xlabel="x", ylabel="y")

for lr in learning_rates
    plot!(p2, x_vec, preds_2[lr], label="lr=$(lr)", linewidth=3)
end

savefig(p2, "learningrate_2HL_Julia.png")
display(p2)


In [ ]:
# 3 hidden layers
preds_3 = Dict{Float64,Vector{Float64}}()

for lr in learning_rates
    nn = build_nn2(1, 50, 1, 3)
    train2!(nn, X, Y, :tanh; epochs=500, lr=lr)
    A, _ = forward2(nn, X, :tanh)
    preds_3[lr] = vec(A[end])
end

p3 = scatter(x_vec, y, label="Data",
             title="3 Hidden Layers - Diferentes Learning Rates",
             xlabel="x", ylabel="y")

for lr in learning_rates
    plot!(p3, x_vec, preds_3[lr], label="lr=$(lr)", linewidth=3)
end

savefig(p3, "learningrate_3HL_Julia.png")
display(p3)


Los resultados muestran que el learning rate intermedio, especialmente 0.01, ofrece el mejor balance entre velocidad y estabilidad. Los learning rates muy pequeños (0.0001) aprenden demasiado lento y no capturan bien el patrón. Los learning rates grandes (0.1) producen resultados inestables o ajustes incorrectos.
